In [1]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import sklearn.linear_model as sl
import scipy.stats as st
%load_ext cython

ncaa = pd.read_csv("http://www4.stat.ncsu.edu/~boos/var.select/ncaa.data2.txt", 
                   delim_whitespace = True)
x = ncaa.ix[:,:-1]
y = ncaa.ix[:,-1]
x.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19
0,13,17,9,15,28.0,0,-1.14045,3.660,4.490,3409,65.8,18,81,42.2,660000,77,100,59,1
1,28,20,32,18,18.4,18,-0.13719,2.594,3.610,7258,66.3,17,82,40.5,150555,88,94,41,25
2,32,20,20,20,34.8,18,1.55358,2.060,4.930,6405,75.0,19,71,46.5,415400,94,81,25,36
3,32,21,24,21,14.5,20,2.05712,2.887,3.876,18294,66.0,16,84,42.2,211000,93,88,26,13
4,24,20,16,20,21.8,13,-0.77082,2.565,4.960,8259,63.5,16,91,41.2,44000,90,92,32,31


In [2]:
pval = np.array([  1.11022302e-16,   6.95109478e-05,   1.15845889e-02,
          5.29907450e-03,   2.48342482e-03,   4.33151925e-02,
          5.27341418e-02,   1.05631519e-01,   8.26270613e-02,
          5.36356789e-02,   2.34958569e-01,   2.86440303e-01,
          3.16318194e-01,   2.69726331e-01,   4.95325787e-01,
          6.32648734e-01,   7.05641795e-01,   8.60540370e-01,
          9.03197593e-01])

In [3]:
%%cython -a
import cython
import numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def helper(double[:] pv_orig, int m1):
    cdef int i,j
    cdef double[:] pvm = np.zeros(m1)
    cdef double[:] alpha = np.zeros(m1+1)
    cdef double[:] alpha2 = np.zeros(m1)
    cdef double[:] ghat2 = np.zeros(m1)
    cdef double[:] S = np.zeros(m1+1)
    cdef double[:] ghat = np.zeros(m1+1)
    
    alpha[0] = 0
    for i in range(0, m1):
        pvm[i] = max(pv_orig[0:(i+1)])
        
    # calculate model size
    with cython.nogil:
        for i in range(1, m1+1):
            alpha[i] = pvm[i-1]
            alpha2[i-1] = pvm[i-1] - 0.0000001
    return (np.array(pvm), np.array(alpha), np.array(alpha2))

In [4]:
%%cython -a
import cython
import numpy as np
from cython.parallel import parallel, prange

@cython.boundscheck(False)
@cython.wraparound(False)
def helper2(double[:] S, double[:] alpha, double[:] alpha2, int m1, int m, int ng):
    cdef int i
    cdef double[:] ghat2 = np.zeros(m1)
    cdef double[:] ghat = np.zeros(m1+1)
        
    with cython.nogil:
        for i in range(1, m1+1):
            ghat[i] = (m - S[i])*alpha[i]/(1 + S[i])
            ghat2[i-1] = (m-S[i-1])*alpha2[i-1]/(1+S[i-1])

    return (np.array(ghat2), np.array(ghat))

In [5]:
def reg_subset(x, y):
    
    lm = sl.LinearRegression()
    
    (n,m) = x.shape
    in_ = []
    out_ = list(range(m))
    rss = np.zeros(m+1)
    rss[0] = sum((y-np.mean(y))**2)
    if(m>=n): ml = n-5
    else: ml = m 

    for pi in range(m):
        rss_find = []
        for i in out_:
            fit_X = pd.DataFrame(x.ix[:, in_ + [i]])
            lm.fit(fit_X, y)
            pred = lm.predict(fit_X)
            rss_find.append(sum((pred-y)**2))
        min_idx = np.argmin(rss_find)
        min_var = out_[min_idx]
        rss[pi+1] = np.min(rss_find)
        in_.append(min_var)
        del out_[min_idx]
    
    in_ = np.array(in_)
    in_var = x.columns[in_]
    vm = np.array(range(ml))
    pv_org = 1 - st.f.cdf((rss[vm] - rss[vm+1])*(n-(vm+2))/rss[vm+1],
                     1,n-(vm+2))
    return (in_,np.array(in_var),rss, pv_org)

In [6]:
def fsr_fast_pv(pv_orig, m, gam0 = 0.05, digits = 4, printout = False, plot = False):
    m1 = len(pv_orig)
    pvm = np.zeros(m1)

    # create monotone p-values
    for i in range(0, m1):
        pvm[i] = np.max(pv_orig[0:(i+1)])
    alpha = np.concatenate([np.array([0]), pvm])
    ng = len(alpha)
    
    # calculate model size
    S = np.zeros(ng)
    for j in range(1, ng):
        S[j] = sum(pvm <= alpha[j])
        
    # calculate gamma hat
    ghat = (m - S)*alpha/(1 + S)
    
    # add additional points to make jumps
    alpha2 = alpha[1:ng] - 0.0000001
    ghat2 = (m - S[0:(ng - 1)])*alpha2/(1 + S[0:(ng - 1)])
    zp = pd.DataFrame({'a': np.concatenate([alpha, alpha2]), 'g': np.concatenate([ghat, ghat2])})
    zp.sort_values(by =['a', 'g'], ascending = [True, False], inplace = True)
    
    # largest gamma hat and index
    gamma_max = np.argmax(zp['g'])
    
    alpha_max = zp['a'][gamma_max]

    # model size with ghat just below gam0
    ind = np.logical_and(ghat <= gam0, alpha <= alpha_max)*1
    Sind = S[np.max(np.where(ind > 0))]
    
    # calculate alpha_F
    alpha_fast = (1 + Sind)*gam0/(m - Sind)
    
    # size of model no intercept
    size1 = sum(pvm <= alpha_fast)
    
    # generate plot
    if plot==True:
        plt.plot(zp['a'], zp['g'], marker = 'o', markersize = 6)
        plt.ylabel('Estimated Gamma')
        plt.xlabel('Alpha')
        pass

    df1 = pd.DataFrame({'pval': pv_orig, 'pvmax': pvm, 'ghigh': ghat2, 'glow': ghat[1:ng]}, columns = ['pval', 'pvmax', 'ghigh', 'glow'])
    df2 = pd.DataFrame({'m1': m1, 'm': m, 'gam0': gam0, 'size': size1, 'alphamax': alpha_max, 'alpha_fast': alpha_fast}, columns = ['m1', 'm', 'gam0', 'size', 'alphamax', 'alpha_fast'], index=[0])
    if printout == True:
        print(df1,df2)
    return(np.round(df1, digits), np.round(df2, digits), ghat)

In [9]:
def fsr_fast_pv_cython(pv_orig, m, gam0 = 0.05, digits = 4, printout = False, plot = False):
    m1 = len(pv_orig)
    ng = m1+1
    (pvm,alpha,alpha2) = helper(pv_orig, m1)
    S = np.zeros(ng)
    for j in range(1, ng):
        S[j] = sum(pvm <= alpha[j])

    # calculate gamma hat
    (ghat2,ghat) = helper2(S,alpha,alpha2,m1,m,ng)
    #ghat = (m - S)*alpha/(1 + S)
    #ghat2 = (m - S[0:(ng - 1)])*alpha2/(1 + S[0:(ng - 1)])
    zp = pd.DataFrame({'a': np.concatenate([alpha, alpha2]), 'g': np.concatenate([ghat, ghat2])})
    zp.sort_values(by =['a', 'g'], ascending = [True, False], inplace = True)
    
    # largest gamma hat and index
    gamma_max = np.argmax(zp['g'])
    
    alpha_max = zp['a'][gamma_max]

    # model size with ghat just below gam0
    ind = np.logical_and(ghat <= gam0, alpha <= alpha_max)*1
    Sind = S[np.max(np.where(ind > 0))]
    
    # calculate alpha_F
    alpha_fast = (1 + Sind)*gam0/(m - Sind)
    
    # size of model no intercept
    size1 = sum(pvm <= alpha_fast)
    
    # generate plot
    if plot==True:
        plt.plot(zp['a'], zp['g'], marker = 'o', markersize = 6)
        plt.ylabel('Estimated Gamma')
        plt.xlabel('Alpha')
        pass

    df1 = pd.DataFrame({'pval': pv_orig, 'pvmax': pvm, 'ghigh': ghat2, 'glow': ghat[1:ng]}, columns = ['pval', 'pvmax', 'ghigh', 'glow'])
    df2 = pd.DataFrame({'m1': m1, 'm': m, 'gam0': gam0, 'size': size1, 'alphamax': alpha_max, 'alpha_fast': alpha_fast}, columns = ['m1', 'm', 'gam0', 'size', 'alphamax', 'alpha_fast'], index=[0])
    if printout == True:
        print(df1,df2)
    return(np.round(df1, digits), np.round(df2, digits), ghat)

In [33]:
%timeit -r3 -n10 fsr_fast_pv(pval,x.shape[1])

10 loops, best of 3: 8.17 ms per loop


In [34]:
%timeit -r3 -n10 fsr_fast_pv_cython(pval,x.shape[1])

10 loops, best of 3: 7.17 ms per loop
